In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import OneHotEncoder
import joblib

In [2]:
df = pd.read_csv('train_sequences.csv')
df[df['temporal_cutoff'] <= '2022-05-27'].shape, df[df['temporal_cutoff'] > '2022-05-27'].shape

((692, 5), (152, 5))

# Criacao da base de dados

In [32]:
def monta_base(sequencias, labels, define_encoder = False):
    base_seqs = pd.read_csv(sequencias)
    base_labels = pd.read_csv(labels)

    base_seqs["resname"] = base_seqs["sequence"].apply(lambda x: list(x))
    base_seqs = base_seqs[["target_id", "sequence", "resname"]].explode("resname", ignore_index=True)
    base_seqs["resid"] = base_seqs.groupby("target_id").cumcount() + 1
    base_seqs["target_num"] = base_seqs["target_id"] + "_" + base_seqs["resid"].astype(str)
    
    base_set = pd.merge(base_seqs[["target_num", "sequence", 'target_id']], base_labels, left_on="target_num", right_on="ID").drop("target_num", axis=1)


    base_set = base_set.dropna(how='any', axis=0).reset_index().drop('index', axis=1)

    base_set['first'] = (base_set['ID'] == base_set['ID'].iloc[0]).astype(int)
    base_set['last'] = (base_set['ID'] == base_set['ID'].iloc[-1]).astype(int)
    base_set = base_set.sort_values(by=['target_id', 'resid'])
    base_set['lead1'] = base_set['resname'].shift(-1)
    base_set['lead2'] = base_set['resname'].shift(-2)
    base_set['lead3'] = base_set['resname'].shift(-3)
    base_set['lag1'] = base_set['resname'].shift(1)
    base_set['lag2'] = base_set['resname'].shift(2)
    base_set['lag3'] = base_set['resname'].shift(3)
    base_set['lag_coord_x'] = base_set['x_1'].shift(1).fillna(0)
    base_set['lag_coord_y'] = base_set['y_1'].shift(1).fillna(0)
    base_set['lag_coord_z'] = base_set['z_1'].shift(1).fillna(0)


    colunas_one_hot = ['resname', 'lead1', 'lead2', 'lead3', 'lag1', 'lag2', 'lag3']
    df_one_hot = base_set[colunas_one_hot]
    df_one_hot['ID'] = '0'
    colunas_one_hot.append('ID')

    if define_encoder:
        enc = OneHotEncoder(handle_unknown='ignore')
        codificados = enc.fit(df_one_hot)
        joblib.dump(enc, "onehot_encoder.pkl")

        
    enc = joblib.load("onehot_encoder.pkl")
    codificados = enc.transform(df_one_hot).toarray()

    renames = []
    for coluna, cats in zip(colunas_one_hot, enc.categories_):
        # print(coluna, cat)
        for cat in cats:
            if cat is None:
                cat = 'None'
            renames.append(coluna + '_' + cat)

    codificados = pd.DataFrame(codificados, columns=renames).iloc[:, :-1]

    base_set = pd.concat([base_set, codificados], axis=1)

    cols = renames[:-1]
    cols.extend(['first', 'last', 'lag_coord_x', 'lag_coord_y', 'lag_coord_z'])

    X = base_set[cols].fillna(99)
    y = base_set[['x_1', 'y_1', 'z_1']]

    # base_modelo = pd.DataFrame({
    #     'features': X.apply(tuple, axis=1),
    #     'prev_coords': base_set[['lag_coord_x', 'lag_coord_y', 'lag_coord_z']].apply(tuple, axis=1),
    #     'true_coords': y.apply(tuple, axis=1)
    # })

    return X, y, base_set

In [33]:
X_train, y_train, base_total = monta_base("train_sequences.csv", "train_labels.csv", False)
X_val, y_val, _ = monta_base("validation_sequences.csv", "validation_labels.csv", False)

C:\Users\pedro\AppData\Local\Temp\ipykernel_19208\2981791205.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hot['ID'] = '0'
C:\Users\pedro\AppData\Local\Temp\ipykernel_19208\2981791205.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_one_hot['ID'] = '0'


# Rede Neural

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
class RNA3DDataset_new(Dataset):
    def __init__(self, X, Y):
        self.X = torch.tensor(X, dtype=torch.float32).to(device)
        self.Y = torch.tensor(Y, dtype=torch.float32).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

dataloader_train = torch.utils.data.DataLoader(RNA3DDataset_new(X_train.values, y_train.values), batch_size=32, shuffle=True)
dataloader_val = torch.utils.data.DataLoader(RNA3DDataset_new(X_val.values, y_val.values), batch_size=32, shuffle=True)


class RNA3DModel_new(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(RNA3DModel_new, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.unsqueeze(1)  # Garante que tem (batch, seq_len, input_size)
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Última saída da sequência
        return x


In [36]:
def predict_sequence(model, X_initial, seq_length):
    model.eval()
    predictions = []
    last_coord = np.array([0, 0, 0])  # Primeira entrada começa com (0,0,0)
    
    for i in range(seq_length):
        X_input = np.hstack([X_initial[i], last_coord])  # Concatenando features + coord anterior
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

        with torch.no_grad():
            pred = model(X_tensor).cpu().numpy().flatten()
        
        predictions.append(pred)
        last_coord = pred  # Atualiza para a próxima entrada
    
    return np.array(predictions)


In [ ]:

# Função de treinamento
def train_model(model, criterion, optimizer, dataloaders, dataset_sizes, scheduler=None, num_epochs=25, patience=15):
    best_loss = float('inf')
    epochs_no_improve = 0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            is_train = phase == 'train'
            model.train() if is_train else model.eval()
            
            running_loss = 0.0
            progress_bar = tqdm(dataloaders[phase], desc=f'{phase.capitalize()}')
            
            for batch_X, batch_Y in progress_bar:
                batch_X = batch_X.to(device, dtype=torch.float32)
                batch_Y = batch_Y.to(device, dtype=torch.float32)
                print(batch_X.shape)
                with torch.set_grad_enabled(is_train):
                    output = model(batch_X)
                    loss = criterion(output, batch_Y)
                    
                    if is_train:
                        optimizer.zero_grad()
                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                        optimizer.step()
                    
                running_loss += loss.item() * batch_X.size(0)
                current_loss = running_loss / dataset_sizes[phase]
                progress_bar.set_postfix({'loss': current_loss})
            
            epoch_loss = running_loss / dataset_sizes[phase]
            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f}')
            
            if phase == 'val' and scheduler:
                scheduler.step(epoch_loss)
            
            if phase == 'val':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    epochs_no_improve = 0
                    torch.save(model.state_dict(), 'best_model.pt')
                else:
                    epochs_no_improve += 1
        
        if epochs_no_improve >= patience:
            print(f'Early stopping! No improvement in validation loss for {patience} epochs.')
            print(f'Best Loss: {best_loss}')
            break
    
    return model

# Criando DataLoaders e scheduler
batch_size = 1
fine_dataloaders = {
    'train': dataloader_train,
    'val': dataloader_val,
}

fine_dataset_sizes = {
    'train': len(dataloader_train),
    'val': len(dataloader_val)
}

# Instanciando o modelo
model = RNA3DModel_new(input_size=46, hidden_size=64, output_size=3).to(device)

# Função de perda e otimizador
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


l2_lambda = 0.3
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)#, weight_decay=l2_lambda)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=30, verbose=True)

# Treinando o modelo
# model = train_model(model, criterion, optimizer, fine_dataloaders, fine_dataset_sizes, scheduler=scheduler, num_epochs=500, patience=500)


In [73]:
torch.save(model.state_dict(), 'best_model.pt')

In [42]:
model = RNA3DModel_new(input_size=46, hidden_size=64, output_size=3).to(device)
model.load_state_dict(torch.load('best_model.pt'))

<All keys matched successfully>

In [29]:
base_total.iloc[:21]

,sequence,target_id,ID,resname,resid,x_1,y_1,z_1,first,last,...,lag2_C,lag2_G,lag2_U,lag2_None,lag3_-,lag3_A,lag3_C,lag3_G,lag3_U,lag3_None
1260,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_1,G,1,35.856998,-10.769000,-7.548,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1261,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_2,G,2,30.230000,-12.075000,-8.614,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1262,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_3,C,3,23.968000,-11.356000,-7.690,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1263,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_4,G,4,19.296000,-9.874000,-4.778,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1264,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_5,U,5,16.362000,-6.047000,-0.706,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1265,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_6,A,6,15.636000,-1.549000,2.463,0,0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1266,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_7,A,7,16.969999,2.893000,4.626,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1267,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_8,G,8,20.391001,6.862000,5.549,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1268,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_9,G,9,24.370001,9.630000,3.348,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1269,GGCGUAAGGAUUACCUAUGCC,17RA_A,17RA_A_10,A,10,26.341999,12.365000,-0.594,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [60]:
def predict_sequence(model, X_initial, seq_length):
    model.eval()
    predictions = []
    last_coord = np.array([0, 0, 0])  # Primeira entrada começa com (0,0,0)
    
    for i in range(seq_length):
        X_input = np.hstack([X_initial[i], last_coord])  # Concatenando features + coord anterior
        X_tensor = torch.tensor(X_input, dtype=torch.float32).unsqueeze(0).to(device)

        with torch.no_grad():
            pred = model(X_tensor).cpu().numpy().flatten()
        
        predictions.append(pred)
        last_coord = pred  # Atualiza para a próxima entrada
    
    return np.array(predictions)

In [62]:
coordenadas = predict_sequence(model, X_train.iloc[:21, :-3].values, 21)

In [74]:
import torch

def compute_d0(L_ref):
    if L_ref >= 30:
        return 0.6 * (L_ref - 0.5) ** 0.5 - 2.5
    elif L_ref < 12:
        return 0.3
    elif 12 <= L_ref <= 15:
        return 0.4
    elif 16 <= L_ref <= 19:
        return 0.5
    elif 20 <= L_ref <= 23:
        return 0.6
    else:  # 24 <= L_ref <= 29
        return 0.7

def kabsch_alignment(P, Q):
    """Aplica o algoritmo de Kabsch para alinhar P (predito) a Q (real)."""
    # Centralizar os pontos no centro de massa
    P_mean = P.mean()
    Q_mean = Q.mean()
    P_centered = P - P_mean
    Q_centered = Q - Q_mean
    
    # Matriz de covariância
    C = torch.matmul(P_centered.T, Q_centered)
    
    # Decomposição SVD para encontrar a melhor rotação
    U, S, V = torch.svd(C)
    d = torch.sign(torch.det(torch.matmul(V, U.T)))  # Ajuste para reflexão
    U_rot = torch.matmul(V, torch.diag(torch.tensor([1, 1, d]))).matmul(U.T)
    
    # Aplicar rotação
    P_aligned = torch.matmul(P_centered, U_rot) + Q_mean  # Transladar de volta
    
    return P_aligned

def tm_score(L_ref, P, Q):
    """Calcula o TM-score entre a estrutura predita (P) e a referência (Q)."""
    P = torch.tensor(P, dtype=torch.float32)
    Q = torch.tensor(Q, dtype=torch.float32)
    P_aligned = kabsch_alignment(P, Q)  # Alinhar estruturas
    d0 = compute_d0(L_ref)
    
    # Distância entre os pontos alinhados
    distances = torch.norm(P_aligned - Q, dim=1)
    
    # Calcular TM-score
    tm = torch.sum(1 / (1 + (distances / d0) ** 2)) / L_ref
    return tm.item()

# Exemplo de uso:
L_ref = 50  # Número de resíduos
P = torch.rand(L_ref, 3) * 10  # Coordenadas preditas aleatórias
Q = torch.rand(L_ref, 3) * 10  # Coordenadas reais aleatórias

score = tm_score(L_ref, P, Q)
print(f"TM-score: {score:.4f}")


TM-score: 0.1144


C:\Users\pedro\AppData\Local\Temp\ipykernel_19208\3820338503.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  P = torch.tensor(P, dtype=torch.float32)
C:\Users\pedro\AppData\Local\Temp\ipykernel_19208\3820338503.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Q = torch.tensor(Q, dtype=torch.float32)


In [75]:
tm_score(21, coordenadas, y_train.iloc[:21].values)

0.001034691696986556

In [76]:
coordenadas

array([[-1.0847820e+00, -2.3159556e+00, -4.1333508e-01],
       [-1.6626283e+00, -3.7375617e+00, -5.6016374e-01],
       [-1.6001683e+00, -4.0288358e+00, -4.7301054e-03],
       [-1.6551267e+00, -3.6416712e+00, -1.8934119e-01],
       [-2.0769048e+00, -4.0940189e+00, -7.8334606e-01],
       [-2.1627364e+00, -4.0642934e+00, -8.6842716e-01],
       [-1.5579461e+00, -4.0688357e+00, -5.8111310e-02],
       [-1.8597060e+00, -4.2635961e+00, -3.6929882e-01],
       [-1.0629088e+00, -3.0347476e+00,  2.6792002e-01],
       [-1.9989663e+00, -4.8247409e+00, -1.7613661e-01],
       [-3.2982969e+00, -6.1738391e+00, -1.3688453e+00],
       [-2.1976633e+00, -4.3325009e+00, -8.5168350e-01],
       [-2.0424757e+00, -4.4308305e+00, -4.4202280e-01],
       [-1.9292163e+00, -4.4445214e+00, -4.9672127e-03],
       [-1.9090673e+00, -4.4092069e+00, -5.5955052e-02],
       [-1.6015040e+00, -3.6968427e+00, -9.1796279e-02],
       [-1.7214345e+00, -4.2612734e+00, -5.1030636e-02],
       [-2.4161596e+00, -4.6377

In [77]:
y_train.iloc[:21].values

array([[ 35.85699844, -10.76900005,  -7.54799986],
       [ 30.22999954, -12.07499981,  -8.61400032],
       [ 23.96800041, -11.35599995,  -7.69000006],
       [ 19.29599953,  -9.8739996 ,  -4.77799988],
       [ 16.36199951,  -6.04699993,  -0.70599997],
       [ 15.63599968,  -1.54900002,   2.46300006],
       [ 16.96999931,   2.89299989,   4.62599993],
       [ 20.39100075,   6.86199999,   5.54899979],
       [ 24.37000084,   9.63000011,   3.34800005],
       [ 26.34199905,  12.36499977,  -0.59399998],
       [ 23.91799927,  16.02300072,  -5.41800022],
       [ 24.93799973,  15.56499958, -11.24300003],
       [ 25.58799934,  10.09500027, -10.00399971],
       [ 28.33300018,   7.8039999 ,  -6.25500011],
       [ 28.9489994 ,   4.83599997,  -0.75599998],
       [ 26.74900055,   1.38999999,   3.28699994],
       [ 24.11899948,  -2.8210001 ,   6.02600002],
       [ 22.77099991,  -7.66499996,   5.35500002],
       [ 22.32999992, -13.6260004 ,   3.10700011],
       [ 25.37299919, -17.35600

In [78]:
import plotly.express as px

In [80]:
preditos = pd.DataFrame(coordenadas, columns=['x', 'y', 'z'])

In [81]:
px.line_3d(preditos, "x", "y", "z", markers="o")

In [82]:
px.line_3d(y_train.iloc[:21], "x_1", "y_1", "z_1", markers="o")